# [Build a Simple LLM Application](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/)
In this quickstart we'll show you how to build a simple LLM application. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

## Using Language Models
First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [11]:
load_dotenv('../')

False

基本のモデル読込。安いgpt3.5にしようとしたらエラー発生。詳しい原因調べていない。

In [6]:
model = ChatOpenAI(model="gpt-4")

プロンプトのメッセージ作成方法とモデル実行

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Japanese"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='こんにちは!', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 20, 'total_tokens': 22}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-97e1e66f-a392-42c8-ac9b-ca0e4bd5552b-0')

## OutputParsers
Notice that the response from the model is an AIMessage. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

We first import the simple output parser.

In [13]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [14]:
result = model.invoke(messages)
print(result)
parser.invoke(result)

content='こんにちは！' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 20, 'total_tokens': 22}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4b2f92d5-1bd4-4672-9aad-9494a1fcdf5c-0'


'こんにちは！'

パーサーと結合したChainを作成し呼出

In [15]:
chain = model | parser
chain.invoke(messages)

'こんにちは！'

## Prompt Templates
Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually it constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

In [16]:
from langchain_core.prompts import ChatPromptTemplate

In [17]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [18]:
result = prompt_template.invoke({"language": "japanese", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into japanese:'), HumanMessage(content='hi')])

In [19]:
result.to_messages()

[SystemMessage(content='Translate the following into japanese:'),
 HumanMessage(content='hi')]

In [20]:
chain = prompt_template | model | parser

In [21]:
chain.invoke({"language": "japanese", "text": "hi"})

'こんにちは'

## Serving with LangServe
Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook or script, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.